# ***Data preparation and augmentation***

**Libraries**

In [ ]:
#Libraries for file handling. 
import os
import shutil
import scipy.io
from os import walk
import pandas as pd

#Libraries for image processing
import cv2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
# To create Folders for different classes.
mat = scipy.io.loadmat('E:\Academic\Btech sem 6\Software Engineering\Project\stanford-cars-dataset\cars_annos.mat')
Names = mat['class_names'][0]

os.chdir(r'E:\Academic\Btech sem 6\Software Engineering\Project\Cars196_resized\training')
for i in range(len(Names)):
    path = str(Names[i][0])
    if not os.path.exists(path):
        os.makedirs(path) 

os.chdir(r'E:\Academic\Btech sem 6\Software Engineering\Project\Cars196_resized\testing')
for i in range(len(Names)):
    path = str(Names[i][0])
    if not os.path.exists(path):
        os.makedirs(path) 

In [ ]:
#Image Processing for training set.
mat = scipy.io.loadmat('E:\Academic\Btech sem 6\Software Engineering\Project\stanford-cars-dataset\cars_train_annos.mat')
Data=mat['annotations'][0]
cnt=1
for data in Data:
    print(cnt)
    cnt+=1
    Class = Names[data[-2][0][0]-1][0]
    x1=data[0][0][0]
    y1=data[1][0][0]
    x2=data[2][0][0]
    y2=data[3][0][0]
    Image_name = data[-1][0]
    path = r'E:\Academic\Btech sem 6\Software Engineering\Project\stanford-cars-dataset\cars_train\cars_train'+'\\'+Image_name
    Dest = r'E:\Academic\Btech sem 6\Software Engineering\Project\Cars196_resized\training'+'\\'+Class+'\\'+Image_name
    img = cv2.imread(path)
    img=img[y1:y2+1,x1:x2+1]
    img = cv2.resize(img, (227,227)) 
    cv2.imwrite(Dest, img) 

#Image Processing for testing set.
Data = pd.read_csv(r'E:\Academic\Btech sem 6\Software Engineering\Project\stanford-cars-dataset\anno_test.csv',header=None)
cnt=1
for i in range(len(Data)):
    print(cnt)
    cnt+=1
    Class = Names[Data[5][i]-1][0]
    x1=Data[1][i]
    y1=Data[2][i]
    x2=Data[3][i]
    y2=Data[4][i]
    Image_name = Data[0][i]
    path = r'E:\Academic\Btech sem 6\Software Engineering\Project\stanford-cars-dataset\cars_test\cars_test'+'\\'+Image_name
    Dest = r'E:\Academic\Btech sem 6\Software Engineering\Project\Cars196_resized\testing'+'\\'+Class+'\\'+Image_name
    img = cv2.imread(path)
    img=img[y1:y2+1,x1:x2+1]
    img = cv2.resize(img, (227,227))
    cv2.imwrite(Dest, img) 

In [ ]:
#To reshuflling training and testing set.
test_path = [x[0] for x in os.walk(r"E:\Academic\Btech sem 6\Software Engineering\Project\sample\testing")]
train_path = [x[0] for x in os.walk(r"E:\Academic\Btech sem 6\Software Engineering\Project\sample\training")]
for i in range(1,len(test_path)):
    files = os.listdir(test_path[i])
    for j in range(0,max(len(files)-20,0)):
        path = os.path.join(test_path[i],files[j])
        files[j]='test'+files[j]
        path1 = os.path.join(test_path[i],files[j])
        os.rename(path,path1)
        shutil.move(path1,train_path[i])

**Image augmentation**

In [ ]:
#Image augmentation for training dataset.
train_path = [x[0] for x in os.walk(r'E:\Academic\Btech sem 6\Software Engineering\Project\Cars196_resized\Training_6')]
for i in range(1,len(train_path)):
    files = os.listdir(train_path[i])
    for j in range(0,len(files)):
        path = os.path.join(train_path[i],files[j])
        datagen = ImageDataGenerator(shear_range = 0.1,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     fill_mode='nearest')
        img = load_img(path)
        x = img_to_array(img) 
        x = x.reshape((1,) + x.shape)
        it = 0
        for batch in datagen.flow(x, batch_size=1,save_to_dir=train_path[i], save_prefix=files[j], save_format='jpg'):
            it += 1
            if it > 1:
                break 